# Initial analysis

## Executive summary

- the train dataset was modified to be used in this study
- not all images seem to be perfectly annotated
- accuracy > XXX % on test set

## More information

**1 & 2 - restructure of the validation and train datasets & creating labels**

As observed and explained on the dataset & paper, the validation dataset was a part of the training set. However this makes no sense. Because of that, the current train will be renamed as train+val and the new train_fixed will be the original dataset without the validation images. This structure fixed the methodological mistake done originally in the dataset (validating in training data).

**3 - img labels**

From the images it is possible to see that the labels and annotations might not be perfect, some seem not to be perfectly annotated, some missing, and such. With that, the model can be only as good as the input data, so that might be a upper limit to its performance.

**4 - training & results**

**(extra) possible improvements**

- increase / improve the dataset
- test better and different models, get larger versions on yolo, other architectures
- improve the pre processing, confirm dataset, test and improve other processes
- improve post-processing, train & evaluate better the _conf_ and _iou_ values to better select all objects avoiding false positives


## imports & configs


In [1]:
#### default imports ####
import numpy as np
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### specific imports ###
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from matplotlib.patches import Ellipse

# forces local code to be reloaded to avoid problems
%load_ext autoreload
%autoreload 2

#### important configs ####
# uses seaborn configs for prettier graphs
sns.set_theme()
# shows thousand separator for values
pd.options.display.float_format = '{:,.2f}'.format
# enable import from src/
sys.path.append('..')  

#### paths ####
# change path to base folder
project_path = "/mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection"

## auxiliar functions


In [2]:
def get_annotations(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    sample_annotations = []

    for neighbor in root.iter("object"):
        label = neighbor.find("name").text
        xmin = int(neighbor.find("bndbox").find("xmin").text)
        ymin = int(neighbor.find("bndbox").find("ymin").text)
        xmax = int(neighbor.find("bndbox").find("xmax").text)
        ymax = int(neighbor.find("bndbox").find("ymax").text)

        #     print(xmin, ymin, xmax, ymax)
        sample_annotations.append([label, xmin, ymin, xmax, ymax])

    return sample_annotations

# 0 - data prep


In [3]:
%cd {project_path}

# # clone dataset in the data/raw folder
# if os.path.exists('data/'):
#     os.removedirs('data/')
# os.makedirs('data/processed')
# os.makedirs('data/raw')

# %cd "data/raw"

# !git clone git@github.com:MahmudulAlam/Complete-Blood-Cell-Count-Dataset.git

# %cd {project_path}

/mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection


/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 1 - validation dataset problem


In [4]:
# # where the images & labels are
# raw_path = "data/raw/Complete-Blood-Cell-Count-Dataset"

# # gets the path for all files
# df_all = pd.DataFrame()
# for dirname, _, filenames in os.walk(raw_path):
#     paths = [dirname + "/" + filename for filename in filenames]
#     folder_name = os.path.split(dirname)[-1]
#     df_all = pd.concat([df_all, pd.DataFrame({"path": paths})], ignore_index=True)

# # transforms to df
# df_all = pd.DataFrame(df_all)

# # also gets the filename
# df_all["filename"] = df_all["path"].apply(lambda s: s.split("/")[-1])

# # and finally check possible extensions
# extensions = df_all["path"].apply(lambda s: s.split(".")[-1])
# extensions.value_counts()

In [5]:
# # creates a reference for the dataset (which folder it is from)
# df_all["dataset"] = df_all["path"].apply(lambda s: s.split("/")[-3])
# df_all["dataset"].value_counts()

In [6]:
# # check if all the files in validation dataset are also in the training one
# for filename in df_all[df_all["dataset"] == "Validation"]["filename"]:
#     if filename not in df_all[df_all["dataset"] == "Training"]["filename"].values:
#         print(filename)

**Conclusion:** Here it is possible to observe that all files in the validation folder are (as explained in the paper & GitHub) duplicated from the training dataset. This utilization is a methodological problem, so will not be used in our study as it is.


# 2 - fixing datasets


## .1 - adapting train, val & test


In [7]:
# # coping all the datasets to the processed folder
# !cp -r data/raw/Complete-Blood-Cell-Count-Dataset/Training data/processed/Training
# !cp -r data/raw/Complete-Blood-Cell-Count-Dataset/Validation data/processed/Validation
# !cp -r data/raw/Complete-Blood-Cell-Count-Dataset/Testing data/processed/Testing
# print('Done!')

In [8]:
# # removing the duplicated images from the training dataset
# removed_files = 0
# for validation_file_path in df_all[df_all["dataset"] == "Validation"]["path"]:
#     validation_file_path_processed_folder = validation_file_path.replace(
#         "raw", "processed"
#     ).replace("Complete-Blood-Cell-Count-Dataset/", "")

#     if os.path.exists(
#         validation_file_path_processed_folder.replace("Validation", "Training")
#     ):
#         os.remove(
#             validation_file_path_processed_folder.replace("Validation", "Training")
#         )
#         removed_files += 1

# print(f"Removed {removed_files} files")

## .2 - verify and recreate df


In [9]:
# # where the images & labels are
# processed_path = "data/processed"

# # gets the path for all files
# df_processed = pd.DataFrame()
# for dirname, _, filenames in os.walk(processed_path):
#     paths = [dirname + "/" + filename for filename in filenames]
#     folder_name = os.path.split(dirname)[-1]
#     df_processed = pd.concat(
#         [df_processed, pd.DataFrame({"path": paths})], ignore_index=True
#     )

# # transforms to df
# df_processed = pd.DataFrame(df_processed)

# # also gets the filename
# df_processed["filename"] = df_processed["path"].apply(lambda s: s.split("/")[-1])

# # and finally check possible extensions
# extensions = df_processed["path"].apply(lambda s: s.split(".")[-1])
# extensions.value_counts()

In [10]:
# processed_images = df_processed[
#     df_processed["filename"].apply(lambda s: s.split(".")[-1] in ["jpg"])
# ]
# # check if all the files in validation dataset are also in the training one
# for filename in df_all[df_all["dataset"] == "Validation"]["filename"]:
#     if filename not in df_all[df_all["dataset"] == "Training"]["filename"].values:
#         print(filename)

## .3 - creating labels


In [11]:
# # definitions for the dataset
# WIDTH = 640
# HEIGHT = 480
# cells_id = {"RBC": 0, "WBC": 1, "Platelets": 2}

# cells_classes = list(cells_id.keys())
# cells_classes
# # saves the dataset into the yolo format
# for i, (index, row) in enumerate(processed_images.iterrows()):
#     # get annotations
#     annotations = get_annotations(
#         row["path"].replace("Images", "Annotations").replace("jpg", "xml")
#     )

#     # get label path
#     label_path = row["path"].replace("Images", "labels").replace("jpg", "txt")

#     # create folders
#     os.makedirs(os.path.split(label_path)[0], exist_ok=True)

#     # save annotations
#     with open(label_path, "w") as file:
#         for label, xmin, ymin, xmax, ymax in annotations:
#             # get the center of the rectangle
#             x_center = (xmin + xmax) / 2
#             y_center = (ymin + ymax) / 2

#             # normalize the values
#             x_center /= WIDTH
#             y_center /= HEIGHT
#             width = (xmax - xmin) / WIDTH
#             height = (ymax - ymin) / HEIGHT

#             # save the values
#             file.write(f"{cells_id[label]} {x_center} {y_center} {width} {height}\n")

# print("done!")

## .4 - saving yaml


In [12]:
# yaml_file = "data/processed/blood_cell_dataset.yaml"

# full_path = "/mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/data/processed"

# train_images_dir = "Training/images/"

# val_images_dir = "Validation/images/"

# test_images_dir = "Testing/images/"

# names_str = ""

# for item in cells_classes:

# names_str = names_str + ", '%s'" % item

# names_str = "names: [" + names_str[1:] + "]"

# with open(yaml_file, "w") as wobj:

# wobj.write("path: %s\n" % full_path)

# wobj.write("train: %s\n" % train_images_dir)

# wobj.write("val: %s\n" % val_images_dir)

# # wobj.write("test: %s\n" % test_images_dir)

# wobj.write("nc: %d\n" % len(cells_classes))

# wobj.write(names_str + "\n")

# 3 - img visualization


In [13]:
# # select only images
# images = df_all[df_all["filename"].apply(lambda s: s.split(".")[-1] in ["jpg"])]

# label_colors = {"RBC": "red", "WBC": "white", "Platelets": "purple"}

# # show 3 images
# fig, ax = plt.subplots(1, 3, figsize=(15, 5))
# for i, (index, row) in enumerate(images.sample(3, random_state=42).iterrows()):
#     # img show
#     img = plt.imread(row["path"])
#     ax[i].imshow(img)
#     ax[i].axis("off")
#     ax[i].set_title(f"{row['dataset']} - {row['filename']}")

#     # get annotations
#     annotations = get_annotations(
#         row["path"].replace("Images", "Annotations").replace("jpg", "xml")
#     )
#     print(annotations)

#     # show annotations
#     for label, xmin, ymin, xmax, ymax in annotations:
#         ax[i].add_patch(
#             # plt.Rectangle(
#             #     (xmin, ymin),
#             #     xmax - xmin,
#             #     ymax - ymin,
#             #     linewidth=2,
#             #     edgecolor=label_colors[label],
#             #     facecolor="none",
#             # )
#             Ellipse(
#                 ((xmin + xmax) / 2, (ymin + ymax) / 2),
#                 xmax - xmin,
#                 ymax - ymin,
#                 linewidth=2,
#                 edgecolor=label_colors[label],
#                 facecolor="none",
#             )
#         )
#         ax[i].text(xmin, ymin, label, fontsize=12, color="k")

**Info:** From the images it is possible to see that the labels and annotations might not be perfect, some seem not to be perfectly annotated, some missing, and such. With that, the model can be only as good as the input data, so that might be a upper limit to its performance.


# 4 - training


In [14]:
from ultralytics import YOLO, settings

# # remove and recreate folders
# if os.path.exists("yolo_data/"):
#     %rm -r yolo_data/
# os.makedirs("yolo_data/datasets", exist_ok=True)
# os.makedirs("yolo_data/weights", exist_ok=True)
# os.makedirs("yolo_data/runs", exist_ok=True)

# Update a setting
settings.update(
    {
        "datasets_dir": "yolo_data/datasets",
        "weights_dir": "yolo_data/weights",
        "runs_dir": "yolo_data/runs",
    }
)

## .1 - initial baseline


In [15]:
# from ultralytics import YOLO

# # Load a model
# model = YOLO("models/yolov8n.pt")  # load a pretrained model (recommended for training)

# results = model.train(
#     data="data/processed/blood_cell_dataset.yaml",
#     epochs=3,
#     imgsz=640,
#     batch=4,
# )

# # show results
# for dataset in ["train", "val"]:
#     print("-" * 30)
#     print(dataset)
#     print("-" * 30)

#     results = model.val(split=dataset)
#     print(results.results_dict)

## .2 - hyperparam tuning


In [16]:
# from ultralytics import YOLO

# # Load a model
# model = YOLO("models/yolov8n.pt")  # load a pretrained model (recommended for training)

# results = model.tune(data="data/processed/blood_cell_dataset.yaml", use_ray=True)

# # show results
# for dataset in ["train", "val"]:
#     print("-" * 30)
#     print(dataset)
#     print("-" * 30)

#     results = model.val(split=dataset)
#     print(results.results_dict)

(_tune pid=36358) New https://pypi.org/project/ultralytics/8.1.34 available 😃 Update with 'pip install -U ultralytics'


2024-03-26 19:06:24,597	ERROR tune_controller.py:1374 -- Trial task failed for trial _tune_1319d_00000
Traceback (most recent call last):
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=36357, ip=172.24.125.125, a

(_tune pid=36358) Ultralytics YOLOv8.1.33 🚀 Python-3.11.8 torch-2.2.1+cu121 CPU (13th Gen Intel Core(TM) i5-13450HX)
(_tune pid=36358) engine/trainer: task=detect, mode=train, model=models/yolov8n.pt, data=data/processed/blood_cell_dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

2024-03-26 19:06:29,326	ERROR tune_controller.py:1374 -- Trial task failed for trial _tune_1319d_00002
Traceback (most recent call last):
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=36700, ip=172.24.125.125, a

(_tune pid=36700) Ultralytics YOLOv8.1.33 🚀 Python-3.11.8 torch-2.2.1+cu121 CPU (13th Gen Intel Core(TM) i5-13450HX) [repeated 3x across cluster]
(_tune pid=36700) engine/trainer: task=detect, mode=train, model=models/yolov8n.pt, data=data/processed/blood_cell_dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

...

2024-03-26 19:06:44,518	ERROR tune_controller.py:1374 -- Trial task failed for trial _tune_1319d_00008
Traceback (most recent call last):
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/branco/miniconda3/envs/bc_detec/lib/python3.11/site-packages/ray/_private/worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=37495, ip=172.24.125.125, a

(_tune pid=37496) New https://pypi.org/project/ultralytics/8.1.34 available 😃 Update with 'pip install -U ultralytics' [repeated 2x across cluster]


2024-03-26 19:06:44,656	ERROR tune.py:1038 -- Trials did not complete: [_tune_1319d_00000, _tune_1319d_00001, _tune_1319d_00002, _tune_1319d_00003, _tune_1319d_00004, _tune_1319d_00005, _tune_1319d_00006, _tune_1319d_00007, _tune_1319d_00008, _tune_1319d_00009]
2024-03-26 19:06:44,657	INFO tune.py:1042 -- Total run time: 24.99 seconds (24.80 seconds for the tuning loop).


------------------------------
train
------------------------------
Ultralytics YOLOv8.1.33 🚀 Python-3.11.8 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset 'coco.yaml' images not found ⚠️, missing path '/mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/yolo_data/datasets/coco/val2017.txt'


 79%|███████▊  | 132M/169M [00:05<00:01, 23.9MB/s] 2024-03-26 19:06:52,661	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
100%|██████████| 169M/169M [00:07<00:00, 22.6MB/s]
Unzipping /mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/yolo_data/datasets/coco2017labels-segments.zip to /mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/yolo_data/datasets/coco...:   1%|          | 1019/122232 [00:08<22:57, 88.02file/s]

## 3 - best model on test


In [18]:
# # Load model
# model = YOLO("yolo_data/runs/detect/train/weights/best.pt")


# # show results
# for dataset in ["train", "val", "test"]:
#     print("-" * 30)
#     print(dataset)
#     print("-" * 30)

#     results = model.val(split=dataset)
#     print(results.results_dict)

------------------------------
train
------------------------------
Ultralytics YOLOv8.1.33 🚀 Python-3.11.8 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/data/processed/Training/labels.cache... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.47it/s]


                   all        240       3140      0.833      0.863      0.895        0.6
                   RBC        240       2641      0.642      0.875      0.816      0.554
                   WBC        240        244      0.961          1      0.992        0.8
             Platelets        240        255      0.898      0.714      0.878      0.445
Speed: 0.5ms preprocess, 4.1ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to yolo_data/runs/detect/val3
{'metrics/precision(B)': 0.833433060250894, 'metrics/recall(B)': 0.8630504884018482, 'metrics/mAP50(B)': 0.8951037886622125, 'metrics/mAP50-95(B)': 0.5997314773492299, 'fitness': 0.6292687084805282}
------------------------------
val
------------------------------
Ultralytics YOLOv8.1.33 🚀 Python-3.11.8 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)


val: Scanning /mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/data/processed/Validation/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.70it/s]


                   all         60        812      0.774      0.885      0.879      0.576
                   RBC         60        700      0.576      0.874      0.795      0.529
                   WBC         60         63      0.957      0.984      0.985      0.749
             Platelets         60         49       0.79      0.796      0.858       0.45
Speed: 1.4ms preprocess, 12.3ms inference, 0.0ms loss, 12.6ms postprocess per image
Results saved to yolo_data/runs/detect/val4
{'metrics/precision(B)': 0.7739581050726904, 'metrics/recall(B)': 0.884777021919879, 'metrics/mAP50(B)': 0.8793331271631004, 'metrics/mAP50-95(B)': 0.5760130940472524, 'fitness': 0.6063450973588373}
------------------------------
test
------------------------------
Ultralytics YOLOv8.1.33 🚀 Python-3.11.8 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)


val: Scanning /mnt/c/Users/nicol/My Drive/personal/coding projects/2024/blood-cell-detection/data/processed/Testing/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.57it/s]


                   all         60        908      0.868      0.891      0.939      0.659
                   RBC         60        792      0.775      0.965       0.95      0.701
                   WBC         60         61      0.964          1      0.995      0.804
             Platelets         60         55      0.867       0.71      0.873      0.472
Speed: 1.8ms preprocess, 7.2ms inference, 0.1ms loss, 18.8ms postprocess per image
Results saved to yolo_data/runs/detect/val5
{'metrics/precision(B)': 0.8683767326536994, 'metrics/recall(B)': 0.8914900080496357, 'metrics/mAP50(B)': 0.9392578897444391, 'metrics/mAP50-95(B)': 0.659138041862484, 'fitness': 0.6871500266506796}
